In [ ]:
import os
import sys
my_path = "/kaggle/input/mlsecu" # path to utils folder
sys.path.append(my_path)
from mlsecu.anomaly_detection_use_case import *
from mlsecu.data_exploration_utils import *
from mlsecu.data_preparation_utils import *
from sklearn.ensemble import IsolationForest
import pandas as pd
import matplotlib.pyplot as plt

### Loading Dataset

In [ ]:
TRAIN_TRANSACTION_FILE_PATH = os.path.join('/kaggle/input/ieee-fraud-detection','train_transaction.csv')

In [ ]:
transaction_df = pd.read_csv(TRAIN_TRANSACTION_FILE_PATH)

In [ ]:
transaction_df.head()

In [ ]:
non_fraud_df = transaction_df[transaction_df['isFraud'] == 0][:9700]
fraud_df = transaction_df[transaction_df['isFraud'] == 1][:300]

In [ ]:
print(f'{non_fraud_df.shape = }')
non_fraud_df.head()

In [ ]:
print(f'{fraud_df.shape = }')
fraud_df.head()

In [ ]:
reduced_transaction_df = pd.concat([non_fraud_df, fraud_df], ignore_index=True)
print(f'{reduced_transaction_df.shape = }')
reduced_transaction_df.head()

In [ ]:
column_number = len(reduced_transaction_df.columns)
print(f'{column_number=}')
print(5*'-')

In [ ]:
column_names = list(reduced_transaction_df.columns)
print("column_names")
for col in column_names:
    types = reduced_transaction_df[col].dtypes
    print(col,'has type',types,end = "\t")
    

In [ ]:
dataset_length = len(reduced_transaction_df)
print(f'{dataset_length=}')

7. Control the correct extraction of transaction: number of non-fraudulent transactions, number of fraudulent transactions, rate of fraudulent transactions

In [ ]:
non_fradulent_number = len(reduced_transaction_df[reduced_transaction_df['isFraud'] == 0])
fradulent_number = len(reduced_transaction_df[reduced_transaction_df['isFraud'] == 1])
fraudulent_rate = fradulent_number /(fradulent_number+non_fradulent_number)
print(f'Number of  non-frauds {non_fradulent_number}')
print(f'Number of frauds {fradulent_number}')
print(f'Fraud rate {fraudulent_rate}')

8. Which columns are categories? List them (show all column names); extract and show existing values.

In [ ]:
categorical_columns = ['ProductCD', 'card1', 'card2', 'card3','card4','card5','card6',
                       'addr1', 'addr2',
                       'P_emaildomain','R_emaildomain', 'M1','M2', 'M3', 'M4','M5', 'M6', 'M7', 'M8', 'M9']
print(f'{categorical_columns  =  }')

9. Which columns are numeric? List them (show all column names); extract and show min, max,mean, median and standard deviation values.


In [ ]:
is_num = lambda type_ : pd.api.types.is_numeric_dtype(type_)
numeric_columns = []
for col in column_names:
    if is_num(reduced_transaction_df[col].dtypes):
        numeric_columns.append(col)

print("Numeric columns are", numeric_columns)

10. For each column, print the rate of undefined values (NaN for numeric)

In [ ]:
undefined_rates = reduced_transaction_df.isna().mean()

# Printing the rate of undefined values for each column
for col,rate in zip(reduced_transaction_df.columns,undefined_rates):
    print(col,rate, end='\t')

11. For each numerical column, print the rate of zero (0) value

In [ ]:
zero_rates = (reduced_transaction_df == 0).mean()
for col,rate in zip(reduced_transaction_df.columns,zero_rates):
    print(col,rate, end='\t')


## DataSet visualisation

12. Visualise the dataset using dimensions: 'TransactionAmt', 'card1','addr1'

In [ ]:

def show3D_transation_data(transac_dataset, x_axis_name, y_axis_name, z_axis_name,title='Data visualisation',color='blue'):
    fig = plt.figure(figsize=(10,10))
    fig.set_facecolor('white')
    ax = fig.add_subplot(1,1,1, projection='3d')

    # Utilisez scatter pour créer le nuage de points en 3D
    ax.scatter(transac_dataset[x_axis_name], transac_dataset[y_axis_name], transac_dataset[z_axis_name],color=color)

    # Définissez les étiquettes des axes
    ax.set_xlabel(x_axis_name)
    ax.set_ylabel(y_axis_name)
    ax.set_zlabel(z_axis_name)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()
    return
show3D_transation_data(reduced_transaction_df,'TransactionAmt', 'card1','addr1')

13. Create an alternate visualisation function for visualising fraud entries only, in red

In [ ]:
def show3D_transation_data_fraud_only(transac_dataset, x_axis_name, y_axis_name, z_axis_name):
    show3D_transation_data(transac_dataset[transac_dataset['isFraud'] ==1 ],'TransactionAmt', 'card1','addr1',title='Frauds',color='red')
    
show3D_transation_data_fraud_only(reduced_transaction_df,'TransactionAmt', 'card1','addr1')

## Data Cleaning

14. Perform one-hot encoding of categorical data

In [ ]:
reduced_transaction_df.shape

In [ ]:
categorical_columns

In [ ]:
print(f'{reduced_transaction_df[categorical_columns].shape = }')
one_hot_df = get_one_hot_encoded_dataframe(reduced_transaction_df[categorical_columns]) 
print(f'{one_hot_df.shape = }')
one_hot_df.head()

15. Remove NaN (Not a number) values by imputation of the mean of the column
- Do not forget to transform the resulting data structure in Pandas Dataframe again
- Do not forget to keep the column names for the Dataframe

In [ ]:
transformed_df = remove_nan_through_mean_imputation(one_hot_df)
print(f'{transformed_df.shape = }')
transformed_df.head()

In [ ]:
def generate_transformed_df(dataframe: pd.DataFrame,selected_cols:list[str]):
    one_hot_df = get_one_hot_encoded_dataframe(reduced_transaction_df[selected_cols])
    transformed_df = remove_nan_through_mean_imputation(one_hot_df)
    return transformed_df

16. Control that no NaN value remain in the dataframe

In [ ]:
if np.all(transformed_df.isna().mean() == 0):
    print("No more Nan value in the data frame")

## Outlier detection

Extract outliers using sklearn.ensemble.IsolationForests, using and outliers_fraction = 0.03. Control the numbers of outliers.
*Typs:*
- clf.predict() returns outliers marked as -1
- Add an additional column ‘outliers’ to your dataframe as follows to store outlier status:
- How many outliers found in unsupervised manner are labelled as fraudulent transactions (`isFraud`) ?
- Print fraudulent outliers. What do you observe?

In [ ]:
def get_list_of_if_outliers_alternate(dataframe: pd.DataFrame, outlier_fraction: float) -> list:
    """Extract the list of outliers according to Isolation Forest algorithm"""

    if dataframe is None:
        return None
    # get list of outliers with sklearn
    clf = IsolationForest(contamination=outlier_fraction, random_state=42)
    # one_hot_df = get_one_hot_encoded_dataframe(dataframe)
    # one_hot_df = remove_nan_through_mean_imputation(one_hot_df)
    clf.fit(dataframe)
    res = clf.predict(dataframe)

    return res

In [ ]:
transformed_df = generate_transformed_df(reduced_transaction_df,categorical_columns)
outliers_indexes = get_list_of_if_outliers_alternate(transformed_df,outlier_fraction=0.03)
transformed_df['outliers'] = outliers_indexes


How many outliers found in unsupervised manner are labelled as fraudulent transactions (`isFraud`) ?

In [ ]:
extended_df = pd.concat([reduced_transaction_df,transformed_df.drop(['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2'],axis=1)],axis=1,join='inner')
outliers_matches = len(extended_df[(extended_df['outliers']== -1) & (extended_df['isFraud']== 1)])
print(f'{outliers_matches} outliers found in unsupervised manner are labelled as fraudulent transactions')
extended_df.shape

18. Create yet another visualisation function for visualising IsolationForest outliers entries only, in red
`show3D_transation_data_if_outliers_only(transac_dataset, x_axis_name, y_axis_name,z_axis_name)`

In [ ]:
def show3D_transation_data_if_outliers_only(transac_dataset, x_axis_name, y_axis_name,z_axis_name):
    show3D_transation_data(transac_dataset, x_axis_name, y_axis_name, z_axis_name,title='Isolation forest outliers',color='red')
sub_df = extended_df[extended_df['outliers']== -1]
show3D_transation_data_if_outliers_only(sub_df,'TransactionAmt', 'card1','addr1')

In [ ]:
sub_df